In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet(path='../data/train.parquet', engine='pyarrow')

In [3]:
df.head()

,programevent_id,channel,market,turnover,weekday,period,date,time,participant_a,participant_b
0,33954710,Retail,standard,17795.0,Friday,Evening,2009-09-18,20:30:00,Schalke 04,VfL Wolfsburg
1,33954710,Retail,double-chance,4950.0,Friday,Evening,2009-09-18,20:30:00,Schalke 04,VfL Wolfsburg
2,33954710,Retail,next-point,4211.0,Friday,Evening,2009-09-18,20:30:00,Schalke 04,VfL Wolfsburg
3,33954710,Retail,points-more-less-than,2031.0,Friday,Evening,2009-09-18,20:30:00,Schalke 04,VfL Wolfsburg
4,33954710,Retail,points-more-less-rest,2587.0,Friday,Evening,2009-09-18,20:30:00,Schalke 04,VfL Wolfsburg


In [4]:
features = ['channel', 'market', 'weekday', 'period', 'participant_a', 'participant_b']

In [5]:
df = pd.get_dummies(data=df, prefix='is_', columns=features).drop(labels=['programevent_id', 'date', 'time'], axis=1)

In [6]:
import numpy as np

In [7]:
features = np.delete(arr=df.columns.values, obj=np.where(df.columns.values == 'turnover'))

In [8]:
df.head()

,turnover,is__Online,is__Retail,is__corner-kick-handicap,is__corner-kicks-more-less-rest,is__corner-kicks-more-less-rest-ht,is__corner-kicks-more-less-than,is__correct-score-live,is__custom-place-yes-no,is__custom-point-yes-no,...,is__Hertha BSC,is__Hoffenheim,is__Kaiserslautern,is__RB Leipzig,is__SC Freiburg,is__SC Paderborn,is__Schalke 04,is__VfB Stuttgart,is__VfL Wolfsburg,is__Werder Bremen
0,17795.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,4950.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,4211.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,2031.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,2587.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [9]:
import random as rand
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

In [10]:
# cross-validation testing
split = rand.uniform(0.60, 0.80)

train_size = int(len(df) * split)

train, test = df[0:train_size], df[train_size:len(df)]

In [11]:
# values of features
X = np.array(train[features].values)

# target values
y = list(train['turnover'])

In [12]:
dtr = DecisionTreeRegressor()

# Set the parameters by cross-validation
parameters = [
    {
        'max_features': ['sqrt', 'log2', None],
        'max_depth': range(2,1000),
    }
]

clf = GridSearchCV(dtr, parameters)
clf.fit(X, y)

NameError: name 'r2_score' is not defined

In [16]:
clf.best_params_

{'max_depth': 24, 'max_features': 'log2'}

In [ ]:
sorted(clf.cv_results_.keys())

In [ ]:
# make predictions
pred = clf.predict(test[features])

In [13]:
from sklearn.metrics import r2_score

# summarize the fit of the model
r2_score(test['turnover'].values, pred)

0.0783477763125845

In [14]:
from sklearn.metrics import mean_squared_error

rms = mean_squared_error(test['turnover'].values, pred)
rms

507486108.046618

In [ ]:
results = pd.DataFrame(data=dict(original=test['turnover'], prediction=pred), index=test.index)

In [ ]:
results